In [ ]:
# Import necessary libraries
import numpy as np
from PIL import Image, ImageFilter
from scipy.ndimage import label, find_objects
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage  # Optional, unless specifically required


In [ ]:
# Imports PIL module
from PIL import Image

# open method used to open different extension image file
im = Image.open("../coin-counting/dataset/coins_images/coins_images/all_coins/01207e3d7e.jpg")

# This method will show image in any image viewer
im.resize(250)

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

csv_path = '/Users/chawkibhd/Desktop/dataset/1/coins_count_values.csv'
data = pd.read_csv(csv_path)
base_image_path = '/Users/chawkibhd/Desktop/dataset/1/coins_images/coins_images'
image_data_list = []
for index, row in data.iterrows():
    folder = row['folder']
    image_name = row['image_name']
    coins_count = row['coins_count']
    image_path = os.path.join(base_image_path, folder, image_name)

    try:
        image = Image.open(image_path)
        image_np = np.array(image)
        image_gray = image.convert('L')
        image_gray_np = np.array(image_gray)
        

        image_data_list.append({
            'image_path': image_path,
            'image_np': image_np,
            'image_gray_np': image_gray_np
        })
        

        #print(f"Image path: {image_path}")
        #print(f"Image as NumPy array: {image_np.shape}")
        #print(f"Image in grayscale as NumPy array: {image_gray_np.shape}")
        
    except Exception as e:
        print(f"Erreur lors de la lecture de l'image {image_path}: {e}")

In [ ]:
if image_data_list:
    first_image_data = image_data_list[88]
    print(f"First image path: {first_image_data['image_path']}")
    print(f"First image as NumPy array shape: {first_image_data['image_np'].shape}")
    print(f"First image in grayscale as NumPy array shape: {first_image_data['image_gray_np'].shape}")
    
     # Afficher l'image couleur
    plt.subplot(1, 2, 1)
    plt.imshow(first_image_data['image_np'])
    plt.title('Couleur')
    
    # Afficher l'image en niveaux de gris
    plt.subplot(1, 2, 2)
    plt.imshow(first_image_data['image_gray_np'], cmap='gray')
    plt.title('Niveaux de gris')
    
    plt.show()

In [1]:
import os
import numpy as np
from Classes.FilterModule import FilterModule
from Classes.ImagePreprocessor import ImageProcessor
from PIL import Image


# Define a function to dynamically test the image
def dynamic_filter_image(image_path):
    # Step 1: Preprocess the image using ImageProcessor
    preprocessor = ImageProcessor(image_path)
    
    # Calculate statistics and determine pre-treatment needs
    pre_treatment = preprocessor.determiner_besoins_pretraitement(
        preprocessor.luminosite_moyenne, 
        preprocessor.ecart_type, 
        preprocessor.min_val, 
        preprocessor.max_val, 
        preprocessor.saturation_moyenne
    )
    
    # Apply pre-treatment processes if necessary
    processed_image = preprocessor.traiter_image()

    # Save the intermediate processed image to pass it to FilterModule
    temp_image_path = "temp_processed_image.jpg"
    processed_image.save(temp_image_path)

    # Step 2: Segment the image using Otsu thresholding
    segmented_image_path = preprocessor.segment_image_using_otsu()

    # Step 3: Apply additional filters using FilterModule
    filter_module = FilterModule(segmented_image_path)
    
    # Example: Determine which filters to apply based on custom needs
    filtering_needs = filter_module.determine_filtering_needs(
        preprocessor.luminosite_moyenne,
        preprocessor.ecart_type,
        preprocessor.min_val,
        preprocessor.max_val,
        preprocessor.saturation_moyenne
    )
    
    # Apply the necessary filters
    filtered_image = filter_module.apply_filtering(filter_module.images[0], filtering_needs)
    
    # Convert filtered image back to PIL format and display/save
    final_image = Image.fromarray(filtered_image)
    final_image.show()  # Optionally show the image
    final_image.save("final_filtered_image.jpg")  # Save the final filtered image

    # Clean up temporary files
    os.remove(temp_image_path)

    return "final_filtered_image.jpg"

In [ ]:

def count_coins(image_path):
    # Load the image
    image = Image.open(image_path).convert('L')  # Convert to grayscale

    # Apply a median filter to reduce noise
    image = image.filter(ImageFilter.MedianFilter(size=5))

    # Convert image to numpy array
    image_array = np.array(image)

    # Apply a binary threshold to get a binary image
    threshold = image_array.mean()
    binary_image = image_array > threshold

    # Compute the Euclidean distance transform
    distance = ndimage.distance_transform_edt(binary_image)

    # Find local maxima
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)), labels=binary_image)

    # Perform connected component analysis on the local peaks
    markers, _ = ndimage.label(local_maxi)

    # Apply the watershed algorithm
    labels = watershed(-distance, markers, mask=binary_image)

    # Count unique labels
    coin_count = len(np.unique(labels)) - 1  # Subtract 1 for the background

    return coin_count


In [ ]:
# Usage example:
# dataset\coins_images\coins_images\all_coins\0bcb8457d5.jpg
image_path = 'dataset/coins_images/coins_images/indian_coins/f3035e00dd.jpg'  # Replace with your image path
processed_image_path = dynamic_filter_image(image_path)
# num_coins = count_coins(image_path)
# print(f"Number of coins detected: {num_coins}")

2024-11-30 20:04:38,345 - INFO - Initialisation de ImageProcessor avec l'image : dataset/coins_images/coins_images/all_coins/f3035e00dd.jpg


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\akram\\docs\\Master ISII\\S1\\Introduction au Traitement d’Images [ITI]\\Project\\code\\coin-counting\\dataset\\coins_images\\coins_images\\all_coins\\f3035e00dd.jpg'

In [ ]:
import logging
import cv2

def count_coins(image_path):
    logging.info("Comptage des pièces dans l'image")
    # Charger l'image filtrée en niveaux de gris
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Appliquer un flou pour réduire le bruit
    blurred_image = cv2.GaussianBlur(image, (15, 15), 0)
    # Détecter les cercles en utilisant la transformation de Hough
    circles = cv2.HoughCircles(blurred_image, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30, param1=50, param2=30, minRadius=15, maxRadius=50)

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        logging.info(f"Nombre de pièces détectées : {len(circles)}")
        # Dessiner les cercles détectés sur l'image
        output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        for (x, y, r) in circles:
            cv2.circle(output_image, (x, y), r, (0, 255, 0), 4)
        # Sauvegarder l'image avec les cercles dessinés
        counted_image_path = "counted_coins_image.jpg"
        cv2.imwrite(counted_image_path, output_image)
        logging.info(f"Image avec pièces comptées sauvegardée à : {counted_image_path}")
    else:
        logging.info("Aucune pièce détectée dans l'image")

In [ ]:
import logging
import cv2

def count_coins(image_path):
    logging.info("Comptage des pièces dans l'image")
    # Charger l'image filtrée en niveaux de gris
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Appliquer un flou pour réduire le bruit
    blurred_image = cv2.GaussianBlur(image, (15, 15), 0)
    # Détecter les cercles en utilisant la transformation de Hough
    circles = cv2.HoughCircles(blurred_image, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30, param1=50, param2=30, minRadius=15, maxRadius=50)

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        logging.info(f"Nombre de pièces détectées : {len(circles)}")
        # Dessiner les cercles détectés sur l'image
        output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        for (x, y, r) in circles:
            cv2.circle(output_image, (x, y), r, (0, 255, 0), 4)
        # Sauvegarder l'image avec les cercles dessinés
        counted_image_path = "counted_coins_image.jpg"
        cv2.imwrite(counted_image_path, output_image)
        logging.info(f"Image avec pièces comptées sauvegardée à : {counted_image_path}")
    else:
        logging.info("Aucune pièce détectée dans l'image")

In [ ]:
count_coins("final_filtered_image.jpg")

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Import the classes
from Classes.FilterModuleOpenCv import FilterModule
from Classes.ImagePreprocessorOpenCv import ImageProcessor

def test_image_processing(image_path):
    # Initialize the ImageProcessor to calculate statistics
    processor = ImageProcessor(image_path)
    stats = processor.stats
    print(f"Image Statistics: Luminosity: {stats[0]:.2f}, Std Dev: {stats[1]:.2f}, Min: {stats[2]}, Max: {stats[3]}, Saturation: {stats[4]:.2f}")
    
    # Determine preprocessing needs
    preprocess_needs = processor.preprocess_needs
    print(f"Preprocessing Needs: {preprocess_needs}")
    
    # Preprocess the image
    preprocessed_image = processor.preprocess_image()

    # Initialize the FilterModule for further dynamic filtering
    filter_module = FilterModule(image_path)
    filtering_needs = filter_module.determine_filtering_needs(stats)
    print(f"Filtering Needs: {filtering_needs}")

    # Apply dynamic filters
    for img in filter_module.images:
        processed_image = filter_module.apply_filtering(img, filtering_needs)
        
        # Display the images
        plt.figure(figsize=(10, 5))
        
        # Original Image
        plt.subplot(1, 2, 1)
        plt.title("Original Image")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')

        # Processed Image
        plt.subplot(1, 2, 2)
        plt.title("Processed Image")
        plt.imshow(cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB))
        plt.axis('off')

        plt.show()


In [ ]:
# Test the function with an example image
# Usage example:
# dataset\coins_images\coins_images\all_coins\0bcb8457d5.jpg
image_path = 'dataset/coins_images/coins_images/all_coins/02e080206f.jpg'  # Replace with your image path
test_image_processing(image_path)
count_coins(image_path)